# Agents for Text-to-SQL with Automatic Error Correction

In this example, we will implement an agent that leverages SQL using `smolagents`.

A standard text-to-sql pipeline is brittle, since the generated SQL query can be incorrect. The query could even be incorrect, but not raise an error, instead giving some incorrect/useless outputs without raising an alarm.

Instead, **an agent system is able to critically inspect outputs and decide if the query needs to be changed or not**, thus giving it a hug performance boost.

## Setups

In [ ]:
!pip install -qU sqlalchemy smolagents

## Setup SQL tables

In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, Float, insert, inspect, text

engine = create_engine('sqlite:///:memory:')
metadata_obj = MetaData()

# Create city SQL table
table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column('receipt_id', Integer, primary_key=True),
    Column('customer_name', String(16), primary_key=True),
    Column('price', Float),
    Column('tip', Float)
)
metadata_obj.create_all(engine)

In [ ]:
rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]

for row in rows:
    stmt = insert(receipts).values(**row)

    with engine.begin() as connection:
        cursor = connection.execute(stmt)

We can test if our system works:

In [ ]:
with engine.connect() as conn:
    rows = conn.execute(text("SELECT * from receipts"))

    for row in rows:
        print(row)

## Build our agent

We need to build a tool for our agent to retrieve our SQL table. Our `sql_engine` tool needs the following:
- A docstring with an `Args:` part. This docstring will be parsed to become the tool's `description` attirbute, which will be used as the instruction manual for the LLM powering the agent.
- Type hints for inputs and outputs.

In [ ]:
from smolagents import tool

@tool
def sql_engine(query: str) -> str:
    """Allows you to perform SQL queries on the table.
    Returns a string representation of the result.
    The table is named 'receipts'. Its description is as follows:
        Columns:
        - receipt_id: INTEGER
        - customer_name: VARCHAR(16)
        - price: FLOAT
        - tip: FLOAT

    Args:
        query: The query to perform. This should be correct SQL
    """
    output = ""
    with engine.connect() as conn:
        rows = conn.execute(text(query))
        for row in rows:
            output += '\n' + str(row)

    return output

Now we can create an agent that leverages this tool.

We wil use the `CodeAgent`, which writes actions in code and can iterarte on previous output according to the ReAct framework.

The `llm_engine` is the LLM that powers the agent system. We will use the `HfApiModel` to call LLMs using HuggingFace's Inference API.

In [ ]:
from smolagents import CodeAgent, HfApiModel

agent = CodeAgent(
    tools=[sql_engine],
    model=HfApiModel('meta-llama/Meta-Llama-3-8B-Instruct')
)

In [ ]:
agent.run("Can you give me the name of the client who got the most expensive receipt?")

## Increasing difficulty: Table joins

In [ ]:
# make a second table
table_name = 'waiters'
receipts = Table(
    table_name,
    metadata_obj,
    Column('waiter_id', Integer, primary_key=True),
    Column('waiter_name', String(16), primary_key=True),
)
metadat_obj.create_all(engine)


rows = [
    {"receipt_id": 1, "waiter_name": "Corey Johnson"},
    {"receipt_id": 2, "waiter_name": "Michael Watts"},
    {"receipt_id": 3, "waiter_name": "Michael Watts"},
    {"receipt_id": 4, "waiter_name": "Margaret James"},
]

for row in rows:
    stmt = insert(receipts).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

Now we need to update the `SQLExecutorTool` with this table's description to let the LLM properly leverage information from this table.

In [ ]:
updated_description = """Allows you to perform SQL queries on the table.
Beware that this tool's output is a string representation of the execution output.
It can use the following tables:"""


inspector = inspect(engine)
for table in ['receipts', 'waiters']:
    columns_info = [
        (col['name'], colp['type'])
        for col in inspector.get_columns(table)
    ]

    table_description = f"Table '{table}':\n"

    table_description += "Columns:\n" + '\n'.join([
        f"  - {name}: {col_type}"
        for name, col_type in columns_info
    ])

    updated_description += '\n\n' + table_description

print(updated_description)

Next we need to replace the `updated_description` in the `sql_engine` tool:

In [ ]:
sql_engine.description = updated_description

Since this request is a bit more difficult than the previous one, we will switch to a more powerful LLM, [`Qwen/Qwen2.5-72B-Instruct`](https://huggingface.co/Qwen/Qwen2.5-72B-Instruct):

In [ ]:
agent = CodeAgent(
    tools=[sql_engine],
    model=HfApiModel('Qwen/Qwen2.5-72B-Instruct')
)

In [ ]:
agent.run("Which waiter got more total money from tips?")